In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs
from plotting import newfig, savefig
from mpl_toolkits.mplot3d import Axes3D
import time
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

np.random.seed(1)
tf.random.set_random_seed(1)

In [ ]:
class PhysicsInformedNN:

    def __init__(self, X_u, u, X_f, layers, lb, ub):
        
        self.lb = lb
        self.ub = ub
    
        self.x_u = X_u[:,0:1]
        self.t_u = X_u[:,1:2]
        
        self.x_f = X_f[:,0:1]
        self.t_f = X_f[:,1:2]
        
        self.u = u
        
        self.layers = layers
        self.weights, self.biases = self.initialize_NN(layers)
        
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
        self.x_u_tf = tf.placeholder(tf.float32, shape=[None, self.x_u.shape[1]])
        self.t_u_tf = tf.placeholder(tf.float32, shape=[None, self.t_u.shape[1]])        
        self.u_tf = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
        
        self.x_f_tf = tf.placeholder(tf.float32, shape=[None, self.x_f.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float32, shape=[None, self.t_f.shape[1]])        
                
        self.u_pred = self.net_u(self.x_u_tf, self.t_u_tf) 
        self.f_pred = self.net_f(self.x_f_tf, self.t_f_tf)         
        
        self.loss = tf.reduce_mean(tf.square(self.u_tf - self.u_pred)) + tf.reduce_mean(tf.square(self.f_pred))
         
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})
        
        init = tf.global_variables_initializer()
        self.sess.run(init)
                
    def initialize_NN(self, layers):  
        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0, num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)  
            
        return weights, biases
        
    def xavier_init(self, size):
        
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        
        num_layers = len(weights) + 1
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0         # scaling
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        
        return Y
            
    def net_u(self, x, t):
        
        u = self.neural_net(tf.concat([x,t],1), self.weights, self.biases)
        u = tf.clip_by_value(u, clip_value_min=0, clip_value_max=0.1)
        
        return u
    
    def net_f(self, x,t):
        
        u = self.net_u(x,t)
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        f = u_t + 30 * u_x - 600 * u * u_x          # conservation law here
        
        return f

    
    def callback(self, loss):
        
        print('Loss:', loss)
        
    def train(self):     
        
        tf_dict = {self.x_u_tf: self.x_u, self.t_u_tf: self.t_u, self.u_tf: self.u,
                   self.x_f_tf: self.x_f, self.t_f_tf: self.t_f}                                                                                                                       
        self.optimizer.minimize(self.sess, 
                                feed_dict = tf_dict,         
                                fetches = [self.loss], 
                                loss_callback = self.callback)        
                                    
    def predict(self, X_star):      
        
        u_star = self.sess.run(self.u_pred, {self.x_u_tf: X_star[:,0:1], self.t_u_tf: X_star[:,1:2]})  
        f_star = self.sess.run(self.f_pred, {self.x_f_tf: X_star[:,0:1], self.t_f_tf: X_star[:,1:2]})   
        
        return u_star, f_star

In [ ]:
np.random.seed(9)
tf.set_random_seed(9)

N_u = 1000
N_f = 1
layers = [2, 50, 50, 50, 50, 50, 50, 50, 50, 50, 1]

data = scipy.io.loadmat('../data/lwr.mat')


t = data['tScale'].T.flatten()[:,None]
x = data['xScale'].T.flatten()[:,None]
Exact = np.real(data['k'])       # density k, Exact has 1440 rows (time), 9 columns (locations)

X, T = np.meshgrid(x,t)
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None])) # X_star shape is 25600 * 2, x left t right
u_star = Exact.flatten()[:,None]       # u_star shape is 25600 * 1, flatten to one dimension       

# Domain boundaries
lb = X_star.min(0)
ub = X_star.max(0)     


xx2 = np.hstack((X[:,1:2], T[:,1:2]))
uu2 = Exact[:,1:2]
xx3 = np.hstack((X[:,-2:-1], T[:,-2:-1]))
uu3 = Exact[:,-2:-1]
xx6 = np.hstack((X[:,125:126], T[:,125:126]))
uu6 = Exact[:,125:126]
xx7 = np.hstack((X[:,250:251], T[:,250:251]))
uu7 = Exact[:,250:251]
xx8 = np.hstack((X[:,375:376], T[:,375:376]))
uu8 = Exact[:,375:376]

# X_f_train (1st command) becomes the random numbers between 0 and 1, dimension 10000 * 2
# X_f_train then vertically stack the X_u_train after itself, becomes 10456 * 2
X_u_train = np.vstack([xx2, xx3, xx6, xx7, xx8])
X_f_train = lb + (ub-lb)*lhs(2, N_f)    # Latin-hypercube designs (lhs) function: lhs(n, [samples, criterion, iterations])
X_f_train = np.vstack((X_f_train, X_u_train))
X_f_train = X_u_train
u_train = np.vstack([uu2, uu3, uu6, uu7, uu8])

# 100 points from the 456 training data.
idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)
X_u_train = X_u_train[idx, :]
u_train = u_train[idx,:]

In [ ]:
model = PhysicsInformedNN(X_u_train, u_train, X_f_train, layers, lb, ub)

start_time = time.time()                
model.train()
elapsed = time.time() - start_time                
print('Training time: %.4f' % (elapsed))

u_pred, f_pred = model.predict(X_star) # X_star is the 25600 * 2, x, t coordiates

error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
print('Error u: %e' % (error_u))                     

U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
Error = np.abs(Exact - U_pred)

In [ ]:
class NN:
    def __init__(self, X_u, u, X_f, layers, lb, ub):
        
        self.lb = lb
        self.ub = ub
    
        self.x_u = X_u[:,0:1]
        self.t_u = X_u[:,1:2]
        
        self.x_f = X_f[:,0:1]
        self.t_f = X_f[:,1:2]
        
        self.u = u
        self.layers = layers
        self.weights, self.biases = self.initialize_NN(layers)
        
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
        self.x_u_tf = tf.placeholder(tf.float32, shape=[None, self.x_u.shape[1]])
        self.t_u_tf = tf.placeholder(tf.float32, shape=[None, self.t_u.shape[1]])        
        self.u_tf = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
        
        self.x_f_tf = tf.placeholder(tf.float32, shape=[None, self.x_f.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float32, shape=[None, self.t_f.shape[1]])        
                
        self.u_pred = self.net_u(self.x_u_tf, self.t_u_tf) 
        self.f_pred = self.net_f(self.x_f_tf, self.t_f_tf)         
        
        self.loss = tf.reduce_mean(tf.square(self.u_tf - self.u_pred)) + 0 * tf.reduce_mean(tf.square(self.f_pred))  
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})
        
        init = tf.global_variables_initializer()
        self.sess.run(init)
                
    def initialize_NN(self, layers):  
        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0, num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)  
            
        return weights, biases
        
    def xavier_init(self, size):
        
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        
        num_layers = len(weights) + 1
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0         # scaling
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        
        return Y
            
    def net_u(self, x, t):
        
        u = self.neural_net(tf.concat([x,t],1), self.weights, self.biases)
        u = tf.clip_by_value(u, clip_value_min=0, clip_value_max=0.1)
        
        return u
    
    def net_f(self, x,t):
        
        u = self.net_u(x,t)
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        f = u_t + 30 * u_x - 600 * u * u_x          # conservation law here
        
        return f

    
    def callback(self, loss):
        
        print('Loss:', loss)
        
    def train(self):     
        
        tf_dict = {self.x_u_tf: self.x_u, self.t_u_tf: self.t_u, self.u_tf: self.u,
                   self.x_f_tf: self.x_f, self.t_f_tf: self.t_f}                                                                                                                       
        self.optimizer.minimize(self.sess, 
                                feed_dict = tf_dict,         
                                fetches = [self.loss], 
                                loss_callback = self.callback)        
                                    
    def predict(self, X_star):      
        
        u_star = self.sess.run(self.u_pred, {self.x_u_tf: X_star[:,0:1], self.t_u_tf: X_star[:,1:2]})  
        f_star = self.sess.run(self.f_pred, {self.x_f_tf: X_star[:,0:1], self.t_f_tf: X_star[:,1:2]})   
        
        return u_star, f_star

In [ ]:
model2 = NN(X_u_train, u_train, X_f_train, layers, lb, ub)

start_time = time.time()                
model2.train()
elapsed = time.time() - start_time                
print('Training time: %.4f' % (elapsed))

u_pred2, f_pred2 = model2.predict(X_star) # X_star is the 25600 * 2, x, t coordiates

error_u2 = np.linalg.norm(u_star-u_pred2,2)/np.linalg.norm(u_star,2)
print('Error u: %e' % (error_u2))                     

U_pred2 = griddata(X_star, u_pred2.flatten(), (X, T), method='cubic')
Error = np.abs(Exact - U_pred2)

In [ ]:
fig, ax = newfig(2, 1)
ax.axis('off')

####### Row 0: PIDL: u(t,x) ##################    
gs0 = gridspec.GridSpec(1, 2)
gs0.update(top=1, bottom=1 - 8/20, left=0.15, right=0.85, wspace=0)
ax = plt.subplot(gs0[:, :])
ax.tick_params(axis='both', which='major', labelsize=20)

h = ax.imshow(U_pred.T, interpolation='nearest', cmap='rainbow', 
              extent=[t.min(), t.max(), x.min(), x.max()], 
              origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cax.tick_params(labelsize=20)
fig.colorbar(h, cax=cax)

ax.plot(X_u_train[:,1], X_u_train[:,0], 'kx', markersize = 2, clip_on = False)

ax.set_xlabel('Time $t$ (s)', fontsize = 20)
ax.set_ylabel('Location $x$ (m)', fontsize = 20)
ax.legend(frameon=False, loc = 'best', fontsize = 20)
ax.set_title('PIDL Estimation $\\rho (x,t)$ (veh./m)', fontsize = 20)

####### Row 1: DL: u(t,x) ##################    
gs1 = gridspec.GridSpec(1, 2)
gs1.update(top = 8/20, bottom=0, left=0.15, right=0.85, wspace=0)
ax = plt.subplot(gs1[:, :])
ax.tick_params(axis='both', which='major', labelsize=20)

h = ax.imshow(U_pred2.T, interpolation='nearest', cmap='rainbow', 
              extent=[t.min(), t.max(), x.min(), x.max()], 
              origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cax.tick_params(labelsize=20)
fig.colorbar(h, cax=cax)

ax.plot(X_u_train[:,1], X_u_train[:,0], 'kx', markersize = 2, clip_on = False)

ax.set_xlabel('Time $t$ (s)', fontsize = 20)
ax.set_ylabel('Location $x$ (m)', fontsize = 20)
ax.legend(frameon=False, loc = 'best', fontsize = 20)
ax.set_title('DL Estimation $\\rho (x,t)$ (veh./m)', fontsize = 20)

# savefig('fixed_1000')